What pages are most popular on the app?
When are customers using the app? Think about day of week, month, season, etc.
How do fans use the app on game days? Pick a sport whose season is within the data we have and look up their schedule from 2021 online. Are fans going to the app more on game days? Are there pages that are more frequently used on game days but not on others?
What is the impact of seasonality on the mobile application data? Remember, these are colleges so think about your own college experience.
Plus any other insights you can find!


Field	Data Type	Description

org_mnemonic	categorical	Identifier of the organization this data comes from

fan_id	UUID	Unique identifier of an individual fan

action_date	date	The date the fan loaded a given page in the app

uphoria_page	categorical	The name of the page in the app that a fan viewed

uphoria_widget	categorical	The widget on an app page that was loaded

# Mobile Analytics Analysis - Rice University

In [19]:
import pandas as pd
import duckdb

%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [20]:
%sql duckdb:///data/rice.db

# Create table for csv import

In [11]:
%%sql
DROP TABLE IF EXISTS rice_university;
CREATE TABLE IF NOT EXISTS rice_university (
    org_mnemonic VARCHAR,
    fan_id VARCHAR,
    action_date DATE,
    uphoria_page VARCHAR,
    uphoria_widget VARCHAR
);


,Success


In [12]:
%%sql
COPY rice_university FROM 'data/Rice.csv' (HEADER);

,Success


In [13]:
%%sql
SELECT * FROM rice_university LIMIT 5;

,org_mnemonic,fan_id,action_date,uphoria_page,uphoria_widget
0,RICE,0e565c4e-2af6-44e3-a9c8-39f17a6c6fb1,2021-08-26,Splash Page,None
1,RICE,d5dfb4c8-52e1-4dd2-a57e-8cd872d0512a,2021-08-26,Splash Page,None
2,RICE,53ae5bf8-8077-4594-bca6-0bcc218324be,2021-08-26,Tickets,IMAGE_WIDGET
3,RICE,b2935c82-f4c9-4ac3-83ef-91c3ca14454b,2021-08-26,Tickets,IMAGE_WIDGET
4,RICE,b2935c82-f4c9-4ac3-83ef-91c3ca14454b,2021-08-26,Tickets,IMAGE_WIDGET


In [36]:
%%sql
CREATE OR REPLACE VIEW most_popular_pages AS
SELECT
    uphoria_page,
    count(*) AS page_views
FROM 
    rice_university 
GROUP BY
    uphoria_page
ORDER BY 
    page_views desc;

,Success


In [131]:
%%sql
SELECT sum(page_views) FROM most_popular_pages;

,sum(page_views)
0,7449.0


In [130]:
%%sql 
select count(*) from rice_university;

,count_star()
0,7449


In [115]:
%%sql
CREATE OR REPLACE VIEW ru_page_views AS
SELECT 
    action_date,
    count(*) as page_views
FROM
    rice_university
GROUP BY
    action_date
ORDER BY
    page_views DESC

,Success


In [132]:
%%sql 
select sum(page_views) from ru_page_views;

,sum(page_views)
0,7449.0


In [15]:
%%sql
SELECT
    MIN(action_date) AS earliest_date,
    MAX(action_date) AS latest_date
FROM rice_university;

,earliest_date,latest_date
0,2021-08-12,2021-10-25


In [60]:
%%sql 
DROP TABLE IF EXISTS dates;
CREATE TABLE IF NOT EXISTS dates (
    date_actual DATE NOT NULL PRIMARY KEY,
    day_name VARCHAR(9) NOT NULL,
    day_of_week INT NOT NULL,
    day_of_month INT NOT NULL,
    month_actual INT NOT NULL,
    quarter_actual INT NOT NULL,
    year_actual INT NOT NULL,
    is_weekend BOOLEAN NOT NULL,
    is_gameday BOOLEAN NOT NULL,
    month_name VARCHAR(9) NOT NULL
)
;

,Success


In [62]:
%%sql
INSERT INTO dates
WITH date_series AS (
    select * as datum from generate_series('2021-08-01'::date, '2021-10-31'::date, '1 day'::interval) 
)
SELECT
    datum::date AS date_actual,
    dayname(datum) AS day_name,
    extract(dow from datum) AS day_of_week,
    extract(day from datum) AS day_of_month,
    extract(month from datum) AS month_actual,
    extract(quarter from datum) AS quarter_actual,
    extract(year from datum) AS year_actual,
    CASE
        WHEN extract(dow from datum) IN (0, 6) THEN TRUE
        ELSE FALSE
    END AS is_weekend,
    FALSE::BOOLEAN AS is_gameday,
    monthname(datum) AS month_name
FROM date_series;


,Success


# Game lookup table

In [191]:
%%sql
DROP TABLE IF EXISTS game_data;
CREATE TABLE IF NOT EXISTS game_data (
    game_date DATE NOT NULL,
    sport VARCHAR(50) NOT NULL,
    gender CHAR(1) NOT NULL
);

,Success


In [192]:
%%sql
INSERT INTO game_data (game_date, sport, gender)
VALUES
('2021-09-04', 'Football', 'M'),
('2021-09-11', 'Football', 'M'),
('2021-09-18', 'Football', 'M'),
('2021-09-25', 'Football', 'M'),
('2021-10-02', 'Football', 'M'),
('2021-10-16', 'Football', 'M'),
('2021-10-23', 'Football', 'M'),
('2021-08-27', 'Volleyball', 'F'),
('2021-08-28', 'Volleyball', 'F'),
('2021-09-02', 'Volleyball', 'F'),
('2021-09-04', 'Volleyball', 'F'),
('2021-09-07', 'Volleyball', 'F'),
('2021-09-10', 'Volleyball', 'F'),
('2021-09-12', 'Volleyball', 'F'),
('2021-09-16', 'Volleyball', 'F'),
('2021-09-17', 'Volleyball', 'F'),
('2021-09-22', 'Volleyball', 'F'),
('2021-09-25', 'Volleyball', 'F'),
('2021-09-26', 'Volleyball', 'F'),
('2021-10-02', 'Volleyball', 'F'),
('2021-10-03', 'Volleyball', 'F'),
('2021-10-15', 'Volleyball', 'F'),
('2021-10-16', 'Volleyball', 'F'),
('2021-10-19', 'Volleyball', 'F'),
('2021-10-22', 'Volleyball', 'F'),
('2021-10-23', 'Volleyball', 'F'),
('2021-10-29', 'Volleyball', 'F'),
('2021-10-30', 'Volleyball', 'F'),
('2021-08-14', 'Soccer', 'F'),
('2021-08-19', 'Soccer', 'F'),
('2021-08-22', 'Soccer', 'F'),
('2021-08-27', 'Soccer', 'F'),
('2021-08-29', 'Soccer', 'F'),
('2021-09-02', 'Soccer', 'F'),
('2021-09-05', 'Soccer', 'F'),
('2021-09-09', 'Soccer', 'F'),
('2021-09-12', 'Soccer', 'F'),
('2021-09-16', 'Soccer', 'F'),
('2021-09-19', 'Soccer', 'F'),
('2021-09-24', 'Soccer', 'F'),
('2021-10-02', 'Soccer', 'F'),
('2021-10-08', 'Soccer', 'F'),
('2021-10-14', 'Soccer', 'F'),
('2021-10-17', 'Soccer', 'F'),
('2021-10-22', 'Soccer', 'F'),
('2021-10-28', 'Soccer', 'F'),
('2021-09-10', 'Cross Country', 'F'),
('2021-09-25', 'Cross Country', 'F'),
('2021-10-01', 'Cross Country', 'F'),
('2021-10-16', 'Cross Country', 'F'),
('2021-10-30', 'Cross Country', 'F'),
('2021-09-17', 'Swimming & Diving', 'F'),
('2021-10-15', 'Swimming & Diving', 'F'),
('2021-09-02', 'Swimming & Diving', 'F'),
('2021-10-01', 'Tennis', 'F'),
('2021-10-02', 'Tennis', 'F'),
('2021-10-03', 'Tennis', 'F'),
('2021-10-04', 'Tennis', 'F'),
('2021-10-14', 'Tennis', 'F'),
('2021-10-15', 'Tennis', 'F'),
('2021-10-16', 'Tennis', 'F'),
('2021-10-18', 'Tennis', 'F'),
('2021-10-28', 'Tennis', 'F'),
('2021-10-29', 'Tennis', 'F'),
('2021-10-30', 'Tennis', 'F'),
('2021-09-10', 'Cross Country', 'M'),
('2021-09-25', 'Cross Country', 'M'),
('2021-10-15', 'Cross Country', 'M'),
('2021-10-30', 'Cross Country', 'M'),
('2021-09-12', 'Golf', 'M'),
('2021-09-13', 'Golf', 'M'),
('2021-09-14', 'Golf', 'M'),
('2021-10-04', 'Golf', 'M'),
('2021-10-05', 'Golf', 'M'),
('2021-10-11', 'Golf', 'M'),
('2021-10-12', 'Golf', 'M'),
('2021-10-13', 'Golf', 'M'),
('2021-10-30', 'Golf', 'M'),
('2021-10-31', 'Golf', 'M'),
('2021-09-17', 'Tennis', 'M'),
('2021-09-18', 'Tennis', 'M'),
('2021-09-19', 'Tennis', 'M'),
('2021-09-24', 'Tennis', 'M'),
('2021-09-25', 'Tennis', 'M'),
('2021-09-26', 'Tennis', 'M'),
('2021-10-01', 'Tennis', 'M'),
('2021-10-02', 'Tennis', 'M'),
('2021-10-03', 'Tennis', 'M'),
('2021-10-14', 'Tennis', 'M'),
('2021-10-15', 'Tennis', 'M'),
('2021-10-16', 'Tennis', 'M'),
('2021-10-17', 'Tennis', 'M'),
('2021-10-18', 'Tennis', 'M'),
('2021-10-19', 'Tennis', 'M'),
('2021-10-29', 'Tennis', 'M'),
('2021-10-30', 'Tennis', 'M'),
('2021-10-31', 'Tennis', 'M');

,Success


In [193]:
%%sql
SELECT sport, gender from game_data group by sport,gender order by sport;

,sport,gender
0,Cross Country,M
1,Cross Country,F
2,Football,M
3,Golf,M
4,Soccer,F
5,Swimming & Diving,F
6,Tennis,M
7,Tennis,F
8,Volleyball,F


# View showing which sports play on each date

In [194]:
%%sql
CREATE OR REPLACE VIEW rice_university_usage_game_days AS
SELECT
    ru.org_mnemonic,
    ru.fan_id,
    ru.action_date,
    ru.uphoria_page,
    ru.uphoria_widget,
    CASE 
        WHEN gd.sport = 'Football'
        THEN 1
        ELSE 0
    END AS is_football,
    CASE
        WHEN gd.sport = 'Cross Country' and gd.gender = 'M'
        THEN 1
        ELSE 0
    END AS is_male_cross_country,
    CASE 
        WHEN gd.sport = 'Cross Country' and gd.gender = 'F'
        THEN 1
        ELSE 0
    END AS is_female_cross_country,
    CASE 
        WHEN gd.sport = 'Golf'
        THEN 1
        ELSE 0
    END AS is_golf,
    CASE 
        WHEN gd.sport = 'Soccer'
        THEN 1
        ELSE 0
    END AS is_soccer,
        CASE 
        WHEN gd.sport = 'Swimming & Diving'
        THEN 1
        ELSE 0
    END AS is_swimming,
    CASE 
        WHEN gd.sport = 'Tennis' and gd.gender = 'M'
        THEN 1
        ELSE 0
    END AS is_male_tennis,
        CASE 
        WHEN gd.sport = 'Tennis' and gd.gender = 'F'
        THEN 1
        ELSE 0
    END AS is_female_tennis,
    CASE 
        WHEN gd.sport = 'Volleyball'
        THEN 1
        ELSE 0
    END AS is_volleyball,
    day_name,
    month_name,
FROM
    rice_university ru
    JOIN game_data gd
    ON ru.action_date = gd.game_date
    JOIN dates d
    on ru.action_date = d.date_actual;
    

,Success


In [195]:
%%sql
SELECT * FROM rice_university_usage_game_days LIMIT 5;

,org_mnemonic,fan_id,action_date,uphoria_page,uphoria_widget,is_football,is_male_cross_country,is_female_cross_country,is_golf,is_soccer,is_swimming,is_male_tennis,is_female_tennis,is_volleyball,day_name,month_name
0,RICE,b2935c82-f4c9-4ac3-83ef-91c3ca14454b,2021-08-27,Tickets,IMAGE_WIDGET,0,0,0,0,1,0,0,0,0,Friday,August
1,RICE,84ab7322-d6b6-4376-9339-82086f613427,2021-08-27,Splash Page,None,0,0,0,0,1,0,0,0,0,Friday,August
2,RICE,b2935c82-f4c9-4ac3-83ef-91c3ca14454b,2021-08-27,GENERIC_PAGE_2,IMAGE_WIDGET,0,0,0,0,1,0,0,0,0,Friday,August
3,RICE,ca02244f-b141-4d80-9b37-56fe048bfd20,2021-08-29,Splash Page,None,0,0,0,0,1,0,0,0,0,Sunday,August
4,RICE,6f2b355f-31db-4ff3-9966-b9730957c53c,2021-08-29,Tickets,IMAGE_WIDGET,0,0,0,0,1,0,0,0,0,Sunday,August


In [202]:
%%sql
SELECT
    gd.sport,
    gd.gender
FROM
    game_data gd
    JOIN dates d ON gd.game_date = d.date_actual
WHERE
    d.day_name = 'Thursday'
GROUP BY
    gd.sport,
    gd.gender;


,sport,gender
0,Volleyball,F
1,Soccer,F
2,Tennis,M
3,Tennis,F
4,Swimming & Diving,F


In [97]:
%%sql
SELECT
    ru.action_date,
    day_name,
    count(*) as page_views
FROM
    rice_university ru
    JOIN dates d on ru.action_date = d.date_actual
GROUP BY
    ru.action_date,
    day_name
ORDER BY
    page_views DESC;

,action_date,day_name,page_views
0,2021-09-25,Saturday,367
1,2021-10-02,Saturday,309
2,2021-10-25,Monday,300
3,2021-09-11,Saturday,280
4,2021-09-24,Friday,235
...,...,...,...
70,2021-08-14,Saturday,11
71,2021-09-15,Wednesday,10
72,2021-08-15,Sunday,6
73,2021-08-23,Monday,4


In [ ]:
%%sql
select sum(page_views) from 

In [197]:
%%sql
SELECT * FROM game_data where game_date = '2021-09-25';

,game_date,sport,gender
0,2021-09-25,Football,M
1,2021-09-25,Volleyball,F
2,2021-09-25,Cross Country,F
3,2021-09-25,Cross Country,M
4,2021-09-25,Tennis,M


In [99]:
%%sql
SELECT
    day_name,
    count(*) as page_views
FROM
    rice_university_usage_game_days
GROUP BY
    day_name
ORDER BY
    page_views DESC;

,day_name,page_views
0,Saturday,4919
1,Friday,2052
2,Sunday,1069
3,Thursday,951
4,Monday,398
5,Tuesday,393
6,Wednesday,254


In [33]:
%%markdown
# App Usage on Game Days

# App Usage on Game Days


In [35]:
%%sql
SELECT
    action_date,
    uphoria_page,
    count(*) as page_views
FROM
    rice_university_usage_game_days
WHERE
    is_football = 1
GROUP BY
    action_date,
    uphoria_page
ORDER BY
    action_date DESC,
    page_views DESC
;


,action_date,uphoria_page,page_views
0,2021-10-23,Splash Page,219
1,2021-10-23,Tickets,9
2,2021-10-23,Event Day,1
3,2021-10-23,Home,1
4,2021-10-16,Splash Page,187
5,2021-10-16,Tickets,8
6,2021-10-16,More,1
7,2021-10-02,Splash Page,271
8,2021-10-02,Tickets,30
9,2021-10-02,GENERIC_PAGE_2,3


In [44]:
%%sql
select 
    action_date,
    count(*) as page_views
from
    rice_university_usage_game_days
where
    is_football = 1
group by
    action_date
;

,action_date,page_views
0,2021-09-04,198
1,2021-09-11,280
2,2021-09-18,230
3,2021-09-25,367
4,2021-10-02,309
5,2021-10-16,196
6,2021-10-23,230


In [50]:
%%sql
select
    action_date,
    CASE
        WHEN is_football = 1 THEN 'Football'
        WHEN is_female_cross_country = 1 then 'Women Cross Country'
        WHEN is_male_cross_country = 1 then 'Men Cross Country'
        WHEN is_golf = 1 then 'Golf'
        WHEN is_soccer = 1 then 'Soccer'
        WHEN is_swimming = 1 then 'Swimming & Diving'
        WHEN is_tennis = 1 then 'Tennis'
        WHEN is_volleyball = 1 then 'Volleyball'
    END as sport,
    count(*) as page_views
from
    rice_university_usage_game_days
where
    is_football = 0
    and
    action_date = '2021-09-02'
group by
    action_date,
    sport
order by
    action_date desc,
    page_views desc;

,action_date,sport,page_views
0,2021-09-02,Soccer,166
1,2021-09-02,Swimming & Diving,166
2,2021-09-02,Volleyball,166


In [52]:
%%sql
select
    CASE
        WHEN is_football = 1 THEN 'Football'
        WHEN is_female_cross_country = 1 THEN 'Female Cross Country'
        WHEN is_male_cross_country = 1 THEN 'Male Cross Country'
        WHEN is_golf = 1 THEN 'Golf'
        WHEN is_soccer = 1 THEN 'Soccer'
        WHEN is_swimming = 1 THEN 'Swimming & Diving'
        WHEN is_tennis = 1 THEN 'Tennis'
        WHEN is_volleyball = 1 THEN 'Volleyball'
    END as sport,
    count(*) as page_views
from
    rice_university_usage_game_days
group by
    sport
order by
    page_views desc;

,sport,page_views
0,Volleyball,2895
1,Football,1810
2,Soccer,1777
3,Tennis,1073
4,Female Cross Country,829
5,Golf,650
6,Male Cross Country,610
7,Swimming & Diving,392


In [55]:
%%sql
WITH number_of_game_days AS (
    SELECT
        sport,
        count(*) as game_days
    FROM
        game_data
    GROUP BY
        sport
),
number_of_page_views AS (
select
    CASE
        WHEN is_football = 1 THEN 'Football'
        WHEN is_female_cross_country = 1 THEN 'Female Cross Country'
        WHEN is_male_cross_country = 1 THEN 'Male Cross Country'
        WHEN is_golf = 1 THEN 'Golf'
        WHEN is_soccer = 1 THEN 'Soccer'
        WHEN is_swimming = 1 THEN 'Swimming & Diving'
        WHEN is_tennis = 1 THEN 'Tennis'
        WHEN is_volleyball = 1 THEN 'Volleyball'
    END as sport,
    count(*) as page_views
from
    rice_university_usage_game_days
group by
    sport
)
SELECT
    sport,
    page_views / game_days as avg_page_views_per_game_day
FROM
    number_of_page_views
    JOIN number_of_game_days USING (sport)
ORDER BY
    avg_page_views_per_game_day DESC;

,sport,avg_page_views_per_game_day
0,Football,258.571429
1,Volleyball,137.857143
2,Swimming & Diving,130.666667
3,Soccer,98.722222
4,Tennis,97.545455
5,Golf,65.000000


In [ ]:
%%sql
select
    day_name,
    count(*) as number_of_game_days
from
    game_data
    join dates
    on game_data.game_date = dates.date_actual
group by
    day_name
;


,day_name,number_of_game_days
0,Thursday,11
1,Saturday,25
2,Friday,19
3,Tuesday,5
4,Sunday,12
5,Monday,5
6,Wednesday,2


In [77]:
%%sql
select 
    game_date,
    count(*) as number_of_games_per_day
from
    game_data
group by
    game_date
order by
    number_of_games_per_day desc;

,game_date,number_of_games_per_day
0,2021-10-30,5
1,2021-09-25,4
2,2021-10-02,4
3,2021-10-15,4
4,2021-10-16,4
5,2021-09-02,3
6,2021-09-10,3
7,2021-09-12,3
8,2021-08-27,2
9,2021-09-04,2


In [80]:
%%sql
select game_date, count(*) from game_data group by game_date order by count(*) desc;

,game_date,count_star()
0,2021-10-30,5
1,2021-09-25,4
2,2021-10-02,4
3,2021-10-15,4
4,2021-10-16,4
5,2021-09-02,3
6,2021-09-10,3
7,2021-09-12,3
8,2021-08-27,2
9,2021-09-04,2


In [78]:
%%sql
select
    sport,
    count(*) as number_of_games
from
    game_data
group by
    sport;

,sport,number_of_games
0,Tennis,11
1,Cross Country,9
2,Swimming & Diving,3
3,Golf,10
4,Football,7
5,Volleyball,21
6,Soccer,18


In [109]:
%%sql
CREATE OR REPLACE VIEW number_of_sports_per_day AS
SELECT
    game_date,
    count(*) as number_of_games
FROM
    game_data
GROUP BY
    game_date;

,Success


In [116]:
%%sql
SELECT
    action_date,
    page_views,
    day_name,
    number_of_games
FROM
    ru_page_views r
    JOIN number_of_sports_per_day n on r.action_date = n.game_date
    JOIN dates d on r.action_date = d.date_actual
ORDER BY
    page_views DESC;

,action_date,page_views,day_name,number_of_games
0,2021-09-25,367,Saturday,4
1,2021-10-02,309,Saturday,4
2,2021-09-11,280,Saturday,1
3,2021-09-24,235,Friday,1
4,2021-09-18,230,Saturday,1
5,2021-10-23,230,Saturday,2
6,2021-09-22,200,Wednesday,1
7,2021-09-04,198,Saturday,2
8,2021-10-16,196,Saturday,4
9,2021-09-02,166,Thursday,3


In [114]:
%%sql
SELECT
    sport
FROM
    game_data
WHERE
    game_date = '2021-09-25';

,sport
0,Football
1,Volleyball
2,Cross Country
3,Cross Country


In [128]:
%%sql
select
    sum(number_of_games),
    month_name,
    sum(page_views)
from
    number_of_sports_per_day n
    join dates d on n.game_date = d.date_actual
    join ru_page_views r on n.game_date = r.action_date
group by
    month_name

,sum(number_of_games),month_name,sum(page_views)
0,32.0,October,1898.0
1,7.0,August,262.0
2,30.0,September,2867.0


In [133]:
%%sql
select * from ru_page_views;

,action_date,page_views
0,2021-09-25,367
1,2021-10-02,309
2,2021-10-25,300
3,2021-09-11,280
4,2021-09-24,235
...,...,...
70,2021-08-14,11
71,2021-09-15,10
72,2021-08-15,6
73,2021-08-23,4


In [134]:
%%sql
CREATE OR REPLACE VIEW ru_page_views_with_dates AS
select
    action_date,
    page_views,
    day_name,
    month_name
from
    ru_page_views r
    join dates d on r.action_date = d.date_actual;

,Success


In [135]:
%%sql
select sum(page_views) from ru_page_views_with_dates;

,sum(page_views)
0,7449.0


In [136]:
%%sql
select
    sum(page_views),
    month_name
from
    ru_page_views_with_dates
group by
    month_name;


,sum(page_views),month_name
0,804.0,August
1,3896.0,September
2,2749.0,October


In [137]:
804 + 3896 + 2749

7449

In [140]:
%%sql
select
    count(*) as number_of_games,
    month_name
from
    game_data
    join dates d on game_data.game_date = d.date_actual
where
    game_date between '2021-08-12' and '2021-10-25'
group by
    month_name;

,number_of_games,month_name
0,32,October
1,7,August
2,30,September


In [144]:
%%sql
select
    action_date,
    count(*) as page_views
from
    rice_university_usage_game_days
where
    is_football = 1
group by
    action_date;

,action_date,page_views
0,2021-09-04,198
1,2021-09-11,280
2,2021-09-18,230
3,2021-09-25,367
4,2021-10-02,309
5,2021-10-16,196
6,2021-10-23,230


In [146]:
%%sql
with football_page_views as (
    select
        action_date,
        count(*) as page_views
    from
        rice_university_usage_game_days
    where
        is_football = 1
    group by
        action_date
)
select
    AVG(page_views)
from
    football_page_views;

,avg(page_views)
0,258.571429


In [148]:
%%sql
with all_page_views as (
    select
        action_date,
        count(*) as page_views
    from
        rice_university_usage_game_days
    group by 
        action_date
)
select
    AVG(page_views)
from
    all_page_views;

,avg(page_views)
0,244.780488


In [150]:
%%sql
with non_football_page_views as (
select
    action_date,
    count(*) as page_views
from
    rice_university_usage_game_days
where
    is_football = 0
group by
    action_date
)
select
    AVG(page_views)
from
    non_football_page_views;

,avg(page_views)
0,210.923077


In [153]:
%%sql
    select
        action_date,
        uphoria_page,
        count(*) as page_views
    from
        rice_university_usage_game_days
    where
        is_football = 1
    group by
        action_date,
        uphoria_page
    order by
        page_views desc,
        action_date desc;

,action_date,uphoria_page,page_views
0,2021-09-25,Splash Page,299
1,2021-10-02,Splash Page,271
2,2021-09-11,Splash Page,220
3,2021-10-23,Splash Page,219
4,2021-09-18,Splash Page,216
5,2021-10-16,Splash Page,187
6,2021-09-04,Splash Page,171
7,2021-09-25,Tickets,48
8,2021-09-11,Tickets,36
9,2021-10-02,Tickets,30


In [ ]:
%%sql
    select
        uphoria_page,
        count(*) as page_views
    from
        rice_university_usage_game_days
    where
        is_football = 1
    group by
        uphoria_page
    order by
        page_views desc;

,uphoria_page,page_views
0,Splash Page,1583
1,Tickets,158
2,GENERIC_PAGE_2,18
3,Home,13
4,Event Day,11
5,Brand Page,8
6,More,7
7,Event Detail,6
8,Generic Page GENERIC_PAGE_2,2
9,Game Day,2


In [158]:
%%sql
with football_page_views as (
    select
        count(*) as fb_page_views
    from
        rice_university_usage_game_days
    where
        is_football = 1
),
all_page_views as (
    select
        count(*) as all_page_views
    from
        rice_university_usage_game_days
)
select
    fb_page_views / all_page_views as fb_page_view_percentage
from
    football_page_views, all_page_views;

,fb_page_view_percentage
0,0.180351


In [159]:
%%sql
select
    count(*) as number_of_games
from
    rice_university_usage_game_days
group by 
    action_date;

,number_of_games
0,11
1,42
2,3
3,194
4,54
5,55
6,498
7,396
8,116
9,118


In [160]:
7/75

0.09333333333333334

In [162]:
(26523 + 18326 + 16031) / 3

20293.333333333332

In [163]:
%%sql
select
    action_date,
    page_views
from
    ru_page_views
where
    action_date in ('2021-09-11', '2021-09-25', '2021-10-02');

,action_date,page_views
0,2021-09-25,367
1,2021-10-02,309
2,2021-09-11,280


In [164]:
(367 + 309 + 280) / 3

318.6666666666667

In [165]:
%%sql
select sum(page_views) from ru_page_views;

,sum(page_views)
0,7449.0


In [173]:
%%sql
CREATE OR REPLACE VIEW ru_fans AS 
select
    action_date,
    fan_id
from
    rice_university
group by
    action_date,
    fan_id;

,Success


In [174]:
%%sql
select * from ru_fans;

,action_date,fan_id
0,2021-08-26,0e565c4e-2af6-44e3-a9c8-39f17a6c6fb1
1,2021-08-29,111aafc4-ba52-41ea-af50-4f2276a1d3a0
2,2021-08-29,b540a3f5-94e4-4068-a744-bfc5ce83881b
3,2021-08-28,a9a7c4ac-f676-4a65-97b1-dd4380e95705
4,2021-08-28,ca02244f-b141-4d80-9b37-56fe048bfd20
...,...,...
6172,2021-10-23,406170d0-a88c-46af-b94d-e20087f75cab
6173,2021-10-23,69448305-7627-42b9-8e1b-eac737c655cb
6174,2021-10-24,75ce5bc6-9f0d-4176-a5bc-98b1a7fbebd8
6175,2021-10-24,c1452e7f-a30d-47ef-88e6-ac1b50efcaf0


In [175]:
%%sql
select count(*) as number_fans from ru_fans group by fan_id;

,number_fans
0,8
1,20
2,21
3,10
4,14
...,...
950,1
951,1
952,1
953,1


In [179]:
%%sql
select count(distinct fan_id)
from 
    ru_fans
where
    action_date in ('2021-09-04', '2021-09-11', '2021-09-18', '2021-09-25', '2021-10-02', '2021-10-16', '2021-10-23')
group by
    action_date;

,count(DISTINCT fan_id)
0,273
1,171
2,222
3,188
4,221
5,216
6,302


In [188]:
(273 + 171 + 222 + 188 + 221 + 216 + 302) / (64065 + 26253 + 91979 + 18326 + 16031 + 27515 + 24845)

0.0059216248968455175

In [190]:
( 171 + 188 + 221 ) / (26253 + 18326 + 16031)

0.009569377990430622